<a href="https://colab.research.google.com/github/alirezasdg1/Amazon-recommender-system/blob/master/notebooks/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## process text data

In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from google.colab import files
import numpy as np

import re
import urllib
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df_all_15 = pd.read_csv("/content/drive/My Drive/Datasets/df_all_15.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df_all_15.drop(['Unnamed: 0','tech1', 'tech2','Unnamed: 0_x','fit','similar_item','date','details','Unnamed: 0_y','reviewTime','reviewerID','reviewerName','vote','image_y','Total_review_C','Total_review_P'], axis=1,inplace=True)

In [9]:
df_all_15.head(2)

,category,description,title,also_buy,image_x,brand,feature,rank,also_view,main_cat,price,asin,overall,verified,reviewText,summary,unixReviewTime,style
0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,False,If you're interested in exploring any area of ...,Great for exploring the rural parts of the state,1470009600,{'Format:': ' Paperback'}
1,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,4.0,True,"Wonderful, comprehensive atlas. It's a little...",Four Stars,1438646400,{'Format:': ' Paperback'}


In [10]:
df_all_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370260 entries, 0 to 370259
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   category        370260 non-null  object 
 1   description     370260 non-null  object 
 2   title           370252 non-null  object 
 3   also_buy        370260 non-null  object 
 4   image_x         370260 non-null  object 
 5   brand           359110 non-null  object 
 6   feature         370260 non-null  object 
 7   rank            370260 non-null  object 
 8   also_view       370260 non-null  object 
 9   main_cat        369863 non-null  object 
 10  price           305197 non-null  object 
 11  asin            370260 non-null  object 
 12  overall         370260 non-null  float64
 13  verified        370260 non-null  bool   
 14  reviewText      370160 non-null  object 
 15  summary         370177 non-null  object 
 16  unixReviewTime  370260 non-null  int64  
 17  style     

In [16]:
df_all_15 = df_all_15[df_all_15['title'].notnull()]

In [18]:
df_all_15["text"] = df_all_15["description"]+df_all_15["brand"]+df_all_15["feature"]+df_all_15["reviewText"]+df_all_15["summary"]+df_all_15['style']

In [22]:
df_all_15.head(2)

,category,description,title,also_buy,image_x,brand,feature,rank,also_view,main_cat,price,asin,overall,verified,reviewText,summary,unixReviewTime,style,text
0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,False,If you're interested in exploring any area of ...,Great for exploring the rural parts of the state,1470009600,{'Format:': ' Paperback'},['Find your way through New York while hitting...
1,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,4.0,True,"Wonderful, comprehensive atlas. It's a little...",Four Stars,1438646400,{'Format:': ' Paperback'},['Find your way through New York while hitting...


In [38]:
df_all_15['description'] = df_all_15['description'].replace(np.nan, ' ', regex=True)
df_all_15['brand'] = df_all_15['brand'].replace(np.nan, ' ', regex=True)
df_all_15['feature'] = df_all_15['feature'].replace(np.nan, ' ', regex=True)
df_all_15['reviewText'] = df_all_15['reviewText'].replace(np.nan, ' ', regex=True)
df_all_15['summary'] = df_all_15['summary'].replace(np.nan, ' ', regex=True)
df_all_15['style'] = df_all_15['style'].replace(np.nan, ' ', regex=True)

In [39]:
df_all_15["text"] = df_all_15["description"]+df_all_15["brand"]+df_all_15["feature"]+df_all_15["reviewText"]+df_all_15["summary"]+df_all_15['style']

In [ ]:
df_all_15["image_x"].iloc[100]


"['https://images-na.ssl-images-amazon.com/images/I/510IoiKdv-L._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/31wNURrrBSL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41kb57MKTsL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/4152ysfRboL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41A8vr5krnL._SS40_.jpg']"

In [ ]:
df_all_15["image_x"].iloc[1].split(',')[0][2:-1]


'https://images-na.ssl-images-amazon.com/images/I/51eXt4FThxL._SS40_.jpg'

In [ ]:
df_all_15["image_x"].iloc[25678].split(',')[0][2:-1]


'https://images-na.ssl-images-amazon.com/images/I/712AbM5F8oL._SS40_.jpg'

In [ ]:
df_all_15["image_x"].iloc[1].split(',')[0][2:-1]

'https://images-na.ssl-images-amazon.com/images/I/51eXt4FThxL._SS40_.jpg'

In [ ]:
df_all_15["image_x"].iloc[0]

"['https://images-na.ssl-images-amazon.com/images/I/51eXt4FThxL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/31bqMEqwU3L._SS40_.jpg']"

## preprosses text data

In [45]:
def text_process(text):
        
        text = re.sub(r'<[^>]*>', ' ', text).strip()
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\w*\d\w*', '', text)
        text = re.sub('[‘’“”…]', '', text)
        text = re.sub('\n', '', text)
        nopunc = [char for char in text if char not in string.punctuation]

        nopunc = ''.join(nopunc)
        list_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
     
        stemmer_snowball = PorterStemmer()
 
        tokens_stemsnowball = [stemmer_snowball.stem(sent) for sent in list_words]

        return ' '.join(tokens_stemsnowball)


In [46]:
df_all_15['text'] = df_all_15['text'].apply(text_process)

In [47]:
df_all_15.head()

,category,description,title,also_buy,image_x,brand,feature,rank,also_view,main_cat,price,asin,overall,verified,reviewText,summary,unixReviewTime,style,text
0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,False,If you're interested in exploring any area of ...,Great for exploring the rural parts of the state,1470009600,{'Format:': ' Paperback'},garminif your interest explor area new york be...
1,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,4.0,True,"Wonderful, comprehensive atlas. It's a little...",Four Stars,1438646400,{'Format:': ' Paperback'},garminwond comprehens atla littl largefour sta...
2,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Find your way through New York while hitting...,Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,True,"Great product, FAST delivery! Just what we nee...",Super!,1432771200,{'Format:': ' Paperback'},garmingreat product fast deliveri need rv trav...
3,"['Sports & Outdoors', 'Outdoor Recreation', 'C...","[""Rely on delorme ATLAS & gazetteer paper maps...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","['>#84,484 in Office Products (See top 100)', ...",[],Office Products,$21.96,0899333257,5.0,True,I have a Delorme atlas for 11 US states. They'...,Necessary for off grid camping,1505260800,{'Format:': ' Paperback'},garmini delorm atla us state theyr amaz detail...
4,"['Sports & Outdoors', 'Outdoor Recreation', 'C...","[""Rely on delorme ATLAS & gazetteer paper maps...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,Garmin,"['Amazingly detailed and beautifully crafted, ...","['>#84,484 in Office Products (See top 100)', ...",[],Office Products,$21.96,0899333257,5.0,False,The first Delorme atlas and gazetteer I ever p...,Don't explore without it,1470614400,,garminth first delorm atla gazett ever purchas...


In [49]:
df_all_15.drop(['description','brand','feature','reviewText','summary',"style"], axis=1,inplace=True)

In [50]:
df_all_15.head()

,category,title,also_buy,image_x,rank,also_view,main_cat,price,asin,overall,verified,unixReviewTime,text
0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,"121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,False,1470009600,garminif your interest explor area new york be...
1,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,"121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,4.0,True,1438646400,garminwond comprehens atla littl largefour sta...
2,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Delorme New York State Atlas &amp; Gazetteer,"['0899334415', '0899334431', '0899333419', '08...",['https://images-na.ssl-images-amazon.com/imag...,"121,074 in Office Products (","['0528881922', '1569145792', '0899334431', '08...",Office Products,$19.95,0899332757,5.0,True,1432771200,garmingreat product fast deliveri need rv trav...
3,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,"['>#84,484 in Office Products (See top 100)', ...",[],Office Products,$21.96,0899333257,5.0,True,1505260800,garmini delorm atla us state theyr amaz detail...
4,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",Garmin DeLorme Atlas &amp; Gazetteer Paper Map...,"['0899334482', '0899334474', '0899333834', '19...",['https://images-na.ssl-images-amazon.com/imag...,"['>#84,484 in Office Products (See top 100)', ...",[],Office Products,$21.96,0899333257,5.0,False,1470614400,garminth first delorm atla gazett ever purchas...


In [ ]:
df_all_15["text"] = df_all_15["brand"]+df_all_15["feature"]+df_all_15["reviewText"]+df_all_15["summary"]+df_all_15['style']

In [54]:
df_all_15.drop(['unixReviewTime','main_cat'],axis=1,inplace=True)

In [21]:
def nlp_process(df):

    df["text"] = df["description"]+df["brand"]+df["feature"]+df["reviewText"]+df["summary"]+df['style']
        
    df.replace(np.nan, ' ', regex=True)
    df['text'] = df['text'].apply(text_process)

    # vectorizer = TfidfVectorizer(min_df = 3,max_df = 0.8, max_features = 1200)
    # m_nlp = vectorizer.fit_transform(df["text"]).toarray()
    # df_nlp = pd.DataFrame(m_nlp, columns=vectorizer.get_feature_names())
    
    return df